In [1]:
import sys
sys.path.append("/Users/admin/Documents/github/UGFAFAFA/code/")
import logging

In [3]:
def log(msf):
    logging.debug(msf)

In [4]:
#  引入文件
from Tusharedata.daily import load
import pandas as pd

# 加载数据
ma = 30
df = load(code="300059.SZ")
df = df[df["date"] > "20210101"]
df.index= pd.to_datetime(df["date"])
from Analyse.haikui import signal,strategy
signal(df,ma)

# 加载回测系统
from rolltrader.cerebro import Cerebro
cerebro = Cerebro()
cerebro.strategy = strategy
cerebro.data = df
cerebro.log = log
edf = cerebro.run()

df = df.join(edf)
result = df[["date","cash","position"]]
result




2021-05-16 13:45:35,056 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-05-16 13:45:35,057 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("data_cache")
2021-05-16 13:45:35,058 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-05-16 13:45:35,060 INFO sqlalchemy.engine.Engine COMMIT
DEBUG:root:>>>>> load DataCache
2021-05-16 13:45:35,081 INFO sqlalchemy.engine.Engine BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
2021-05-16 13:45:35,092 INFO sqlalchemy.engine.Engine SELECT data_cache.id AS data_cache_id, data_cache."key" AS data_cache_key, data_cache.value AS data_cache_value 
FROM data_cache 
WHERE data_cache."key" = ?
 LIMIT ? OFFSET ?
INFO:sqlalchemy.engine.Engine:SELECT data_cache.id AS data_cache_id, data_cache."key" AS data_cache_key, data_cache.value AS data_cache_value 
FROM data_cache 
WHERE data_cache."key" = ?
 LIMIT ? OFFSET ?
2021-05-16 13:45:35,095 INFO sqlalchemy.engine.Engine [generated in 0.00512s] ('daily.300059.sz', 1, 0)
INFO:sqlalchemy.engine

,date,cash,position
date,,,
2021-01-04,20210104,10000.00,0
2021-01-05,20210105,10000.00,0
2021-01-06,20210106,10000.00,0
2021-01-07,20210107,10000.00,0
2021-01-08,20210108,10000.00,0
...,...,...,...
2021-05-10,20210510,2526.05,200
2021-05-11,20210511,2526.05,200
2021-05-12,20210512,2526.05,200
